In [1]:
from hjst_model.config import DatasetKVSConfig, LayerModelConfig, HierarchicalModelConfig
import numpy as np
import scipy
import nmslib
import pandas as pd
import os
import re
import sentencepiece as spm

import sys 
import time 
import math 
from scipy.sparse import csr_matrix 
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

from collections import Counter
from jstatutree import etypes
from pathlib import Path

#カラム内の文字数。デフォルトは50だった
pd.set_option("display.max_colwidth", 800)
pd.set_option("display.width", 3)

#行数
pd.set_option("display.max_rows", 101)

def coloring(threshold, val):
    ret = []
    for i, v in enumerate(val):
        if not isinstance(v, str) and v < threshold:
            ret[0] = 'background-color: yellow' 
            ret[-1] = 'background-color: yellow'
            ret.append('background-color: yellow')
        else:
            ret.append('background-color: white')
    return ret

def get_caption(query):
    try:
        return testset.get_elem(query+"/{}Caption(1)".format(re.split("\(", os.path.split(query)[1])[0])).text
    except Exception as e:
        return ""

In [3]:
TESTSET = "aichi"
CONF_DIR = './configs'
HMODEL_NAME = 'ttt-as8'

testset_conf = DatasetKVSConfig()
testset_conf.link(os.path.join(CONF_DIR, "dataset.conf"))
testset_conf.change_section(TESTSET)
testset = testset_conf.prepare_dataset(False)

# prepare layer config instance
layerconf = LayerModelConfig()
layerconf.link(os.path.join(CONF_DIR, "layer.conf"))

model_conf = HierarchicalModelConfig()
model_conf.link(os.path.join(CONF_DIR, HMODEL_NAME+'.conf'), create_if_missing=True)
hmodel = model_conf.model_generate()
testset.set_iterator_mode(level='Law', tag=True, sentence=False)
ltag_gen = (ltag for ltag in list(testset))
#hmodel.topk_comptable(testset, '23/230006/0001/Law(1)', 2, 0.2, 'Sentence')

/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TruncatedSVD from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator Pipeline from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  Us

set layer: TfidfLayer 0.0
set layer: TfidfLayer 0.0
set layer: TfidfLayer 0.0


In [11]:
hmodel.layers['Sentence'] = layerconf.load_model('swem_ave_gv_tokai_spm_8000-Sentence')

In [2]:
# データセット変更用
del testset
testset_conf.change_section("tokai")
testset = testset_conf.prepare_dataset(False)
testset.set_iterator_mode(level='Law', tag=True, sentence=False)
ltag_gen = (ltag for ltag in list(testset))

NameError: name 'testset' is not defined

In [9]:
from hjst_model.tokenizer import load_spm
#m = layerconf.load_model('swem_ave_gv_tokai_spm_8000-Sentence')
testset.set_iterator_mode(level='Law', gensim=True)
for td in testset:
    print(td.tags, td.words)
    break

['23/230006/0001/Law(1)'] ['', 'この', '条例', 'は', '、', '地方', '自治', '法', '(', '昭和', '二', '十', '二', '年', '法律', '第', '六', '十', '七', '号', ')', '第', '十', '六', '条', 'の', '規定', 'に', '基き', '、', '条例', '、', '規約', '、', '規則', '、', '教育', '委員', '会', 'を', '除く', '県', 'の', '機関', 'の', '定める', '規則', '及び', 'その他', 'の', '規程', 'で', '公表', 'を', '要する', 'もの', 'の', '公布', '又は', '公表', 'に関し', '必要', 'な', '事項', 'を', '定める', 'こと', 'を', '目的', 'と', 'する', '。', '条例', 'を', '公布', 'しよ', 'う', 'と', 'する', 'とき', 'は', '、', '公布', 'の', '旨', 'の', '前文', '及び', '年月日', 'を', '記入', 'し', 'て', 'その', '末尾', 'に', '知事', 'が', '署名', 'し', 'なけれ', 'ば', 'なら', 'ない', '。', '条例', 'の', '公布', 'は', '、', '愛知', '県', '公報', 'に', '登載', 'し', 'て', '行う', '。', 'ただし', '、', '天災', 'その他', 'やむを得ない', '事情', 'により', '愛知', '県', '公報', 'に', '登載', 'し', 'て', '公布', 'する', 'こと', 'が', 'でき', 'ない', 'とき', 'は', '、', '県庁', '前', 'の', '掲示板', '又は', '公衆', 'の', '見', 'やすい', '場所', 'に', '掲示', 'し', 'て', '行う', 'こと', 'が', 'できる', '。', '前条', 'の', '規定', 'は', '、', '規約', '及び', '規則', 'に', '準用', 'する', '。', '規約'

In [4]:
# Lawレベルに対してインデックス作成
law_tags = list(testset)
_, law_index = hmodel.create_index('Law', law_tags)

In [4]:
# テストレイヤーの読み込み
test_sentence_layers = {l:{'layer': layerconf.load_model(l+'-Sentence'), 'threshold': th} for (l, th) in [
    ('swem_average_tokai_mecab', 0.2),
    ('swem_ave_ft_tokai_spm_8000', 0.2),
    #('swem_ave_ft_tokai_spm_16000', 0.2),
    ('swem_ave_gv_tokai_spm_8000', 0.2),
    #('swem_ave_gv_tokai_spm_16000', 0.2),
    #("swem_ave_ft_tokai_mecab", 0.2),
    #('swem_concat_tokai_mecab', 0.2),
    #('swem_max_tokai_mecab',0.01),
    ('swem_average_tokai_spm_8000',0.2),
    #('swem_concat_tokai_spm_8000',0.2),
    #('swem_average_tokai_spm_16000',0.2),
    #('swem_concat_tokai_spm_16000',0.2),
    #('tf_tokai_spm_8000',0.5),
    #('tfidf_tokai_spm_8000',0.5)
]}
test_sentence_layers["swem_ave_ft_tokai_mecab"] = {"layer": hmodel.layers["Sentence"], 'threshold': 0.2}

In [5]:
# 各テストレイヤーに対してSentenceインデックス作成
testset.set_iterator_mode(level='Sentence', tag=True, sentence=False)
tags = list(testset)
for k in test_sentence_layers.keys():
    print(k)
    if 'index' in test_sentence_layers[k]:
        continue
    test_sentence_layers[k]['vectors'] = np.matrix([test_sentence_layers[k]['layer'][tag] for tag in tags])
    index = nmslib.init(method='hnsw', space='cosinesimil')
    index.addDataPointBatch(test_sentence_layers[k]['vectors'])
    index.createIndex({'post': 2}, print_progress=True)
    test_sentence_layers[k]['index'] = index
ltag_gen = (ltag for ltag in list(testset))

swem_average_tokai_mecab
swem_ave_ft_tokai_spm_8000
swem_ave_gv_tokai_spm_8000
swem_average_tokai_spm_8000
swem_ave_ft_tokai_mecab


In [9]:
# キーワード検索
keyword = "暴力団"
lt = None
testset.set_iterator_mode(level='Law', tag=True, sentence=False)
for ltag in testset:
    if keyword in str(testset.get_elem(ltag)):
        lt = ltag
        print(lt, str(testset.get_elem(lt)))

23/230006/1797/Law(1) 愛知県公安委員会の権限に属する暴力団員による不当な行為の防止等に関する法律に規定する事務の一部を警察本部長等に委任する規則
23/230006/1813/Law(1) 愛知県暴力団排除条例
23/230006/1814/Law(1) 愛知県暴力団排除条例施行規則
23/231002/0999/Law(1) 名古屋市暴力団排除条例
23/232017/0086/Law(1) 豊橋市暴力団排除条例
23/232025/0692/Law(1) 岡崎市暴力団排除条例
23/232033/0700/Law(1) 一宮市暴力団等の排除に関する条例
23/232041/0562/Law(1) 瀬戸市暴力団排除条例
23/232050/0493/Law(1) 半田市暴力団排除条例
23/232068/0614/Law(1) 春日井市暴力団排除条例
23/232076/0498/Law(1) 豊川市暴力団排除条例
23/232084/0069/Law(1) 津島市暴力団排除条例
23/232092/0388/Law(1) 碧南市暴力団排除条例
23/232106/0325/Law(1) 刈谷市暴力団排除条例
23/232114/0797/Law(1) 豊田市暴力団排除条例
23/232122/0380/Law(1) 安城市暴力団排除条例
23/232131/0401/Law(1) 西尾市暴力団排除条例
23/232149/0555/Law(1) 蒲郡市暴力団排除条例
23/232157/0545/Law(1) 犬山市暴力団排除条例
23/232165/0610/Law(1) 常滑市暴力団排除条例
23/232173/0447/Law(1) 江南市暴力団排除条例
23/232190/0528/Law(1) 小牧市暴力団排除条例
23/232203/0089/Law(1) 稲沢市暴力団排除条例
23/232211/0595/Law(1) 新城市暴力団排除条例
23/232246/0460/Law(1) 知多市暴力団排除条例
23/232254/0273/Law(1) 知立市暴力団排除条例
23/232262/0345/Law(1) 尾張旭市暴力団排除条例
23/232271/0570/Law(1) 高浜市暴力団排除条例
23/232289/03

In [5]:
query = "23/230006/1813/Law(1)"
k = 100
root_threshold =  1000
threshold = 0.3
print("query:", testset.get_elem(query))
qlevel = re.split('\(', os.path.split(query)[1])[0]
tidx, distance = law_index.knnQuery(hmodel.layers[qlevel][query], k=k+1)
target_law_tags = [query]
for i, dist in zip(tidx, distance):
    if law_tags[i] == query:
        continue
    if root_threshold and dist > root_threshold:
        print("not enough distance", testset.get_elem(law_tags[i]), dist)
        k -= 1
        continue
    print("add target:", testset.get_elem(law_tags[i]), dist)
    target_law_tags.append(law_tags[i])

query: 愛知県暴力団排除条例
add target: 名古屋市暴力団排除条例 0.14395046
add target: 豊根村暴力団排除条例 0.15286398
add target: 飛島村暴力団排除条例 0.15621924
add target: 長久手市暴力団排除条例 0.15823942
add target: 津島市暴力団排除条例 0.15826392
add target: 半田市暴力団排除条例 0.15994483
add target: 豊橋市暴力団排除条例 0.16114217
add target: 西尾市暴力団排除条例 0.16163337
add target: 北名古屋市暴力団排除条例 0.16167903
add target: 弥富市暴力団排除条例 0.16187716
add target: 日進市暴力団排除条例 0.16248995
add target: 知多市暴力団排除条例 0.16290933
add target: 豊川市暴力団排除条例 0.16468048
add target: 知立市暴力団排除条例 0.16520774
add target: 愛西市暴力団排除条例 0.165784
add target: 豊明市暴力団排除条例 0.1659376
add target: 刈谷市暴力団排除条例 0.1664862
add target: 稲沢市暴力団排除条例 0.16650677
add target: 高浜市暴力団排除条例 0.1666767
add target: 岩倉市暴力団排除条例 0.1669612
add target: 新城市暴力団排除条例 0.16746897
add target: 江南市暴力団排除条例 0.16767025
add target: 田原市暴力団排除条例 0.16790032
add target: 蒲郡市暴力団排除条例 0.16824567
add target: 瀬戸市暴力団排除条例 0.16830319
add target: 清須市暴力団排除条例 0.16831183
add target: 岡崎市暴力団排除条例 0.1687274
add target: みよし市暴力団排除条例 0.16895664
add target: 春日井市暴力団排除条例 0.169505

In [3]:
query = "23/230006/1813/Law(1)"
qlevel = re.split('\(', os.path.split(query)[1])[0]
comp_level = 'Sentence'
qk = 100000
comp_k = 100000
threshold = 0.2
print("query:", testset.get_elem(query))

query: 愛知県暴力団排除条例


In [4]:
result_df = dict()

print('Construct index for', comp_level, 'level')
testset.set_iterator_mode(level=comp_level, tag=True, sentence=False)
tags = list(testset)
qdata_matrix = np.matrix([hmodel.layers[comp_level][tag] for tag in tags])
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(qdata_matrix)
start = time.time()
index.createIndex({'post': 2}, print_progress=True)
end = time.time()
print('ANN', comp_level, 'preparation time %f' % (end - start))

print('Construct index for', qlevel, 'level')
testset.set_iterator_mode(level=qlevel, tag=True, sentence=False)
qtags = list(testset)
print('using', len(qtags), qlevel+'s')
start = time.time()
cdata_matrix = np.matrix([hmodel.layers[qlevel][tag] for tag in qtags])
sindx = NearestNeighbors(n_neighbors=min(qk, cdata_matrix.shape[0]), metric='cosine', algorithm='brute').fit(cdata_matrix)
end = time.time()
print('KNN', comp_level, 'preparation time %f' % (end - start))

Construct index for Sentence level
ANN Sentence preparation time 132.023018
Construct index for Law level
using 20613 Laws
KNN Sentence preparation time 0.044848


In [6]:
# 条文対応表作成特化型検索
start = time.time() 
d, i = sindx.kneighbors(np.matrix(hmodel.layers[qlevel][query]))
print(i)
end = time.time()
qsearch_result =  list(zip(i[0], d[0]))
print('search', len(qsearch_result), qlevel+'s', 'from', len(qtags), qlevel+'s')
print('over all top search time', round(end-start, 3), 'sec')

"""
print('Construct index for', qlevel, 'level')
testset.set_iterator_mode(level=qlevel, tag=True, sentence=False)
qtags = list(testset)
print('using', len(qtags), qlevel+'s')
vectors = np.matrix([self.layers[level][tag] for tag in tags])
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(vectors)
index.createIndex({'post': 2}, print_progress=True)
print(vectors.shape)
"""


print('create', qlevel, 'counter')
qlevel_counter = Counter()
search_time = 0.0
for elem in testset.get_elem(query).depth_first_iteration([getattr(etypes, comp_level)]):
    t = time.time()
    tidx_list, distance_list = index.knnQuery(hmodel.layers[comp_level][elem.code], k=comp_k+1)
    search_time += time.time() - t
    for tidx, distance in zip(tidx_list, distance_list):
        if distance > threshold:
            break
        i = 0
        tag_parts = Path(tags[tidx]).parts
        for p in tag_parts:
            if qlevel in p:
                break
            i += 1
        qlevel_tag = '/'.join(tag_parts[:i+1])
        qlevel_counter[qlevel_tag] = qlevel_counter.get(qlevel_tag, 0) + 1
print('over all bottom up search time', round(search_time, 3), 'sec')
        
target_law_tags = [query]
result_df[comp_k] = pd.DataFrame(columns=['tag', 'name', 'similar_count', 'distance', 'top_down_rank'])
rank = 0
for tag, count in qlevel_counter.most_common(100+1):
    if tag == query:
        continue
    ldist = 'Out of top{}'.format(qk)
    qs_rank = None
    for r, (ltidx, dist) in enumerate(qsearch_result):
        if qtags[ltidx] == tag:
            ldist = dist
            qs_rank = r
            break
    target_law_tags.append(tag)
    rank += 1
    result_df[comp_k].loc[rank] = [tag, testset.get_elem(tag), count, ldist, qs_rank]
result_df[comp_k]

[[  635  1329 20594 ... 16287  2168 20325]]
search 20613 Laws from 20613 Laws
over all top search time 0.11 sec
create Law counter
over all bottom up search time 0.022 sec


,tag,name,similar_count,distance,top_down_rank
1,23/235610/0466/Law(1),設楽町暴力団排除条例,15,0.191837,38
2,23/232017/0086/Law(1),豊橋市暴力団排除条例,15,0.161142,7
3,23/232084/0069/Law(1),津島市暴力団排除条例,15,0.158264,5
4,23/234478/0400/Law(1),武豊町暴力団排除条例,14,0.218262,49
5,23/234419/0384/Law(1),阿久比町暴力団排除条例,14,0.201325,42
6,23/232246/0460/Law(1),知多市暴力団排除条例,14,0.162909,12
7,23/232254/0273/Law(1),知立市暴力団排除条例,14,0.165208,14
8,23/232041/0562/Law(1),瀬戸市暴力団排除条例,14,0.168303,25
9,23/232351/0368/Law(1),弥富市暴力団排除条例,14,0.161877,10
10,23/233421/0551/Law(1),豊山町暴力団排除条例,13,0.206419,46


In [17]:
target_law_tags = ['23/230006/1813/Law(1)', '23/230006/0317/Law(1)']
query = target_law_tags[0]

In [18]:
df = hmodel.comptable(testset, query, target_law_tags[1:], 10, 'Sentence')
display(df.style.apply(lambda x: coloring(threshold, x), axis=1))

In [8]:
columns = []
for k, df in dfdict.items():
    q = (target_law_tags[0], 'Sentence')
    df.columns = pd.MultiIndex.from_tuples([q, (k, 'Sentence'), (k, 'score')])
    print(k)
    if 'spm' in k:
        from hjst_model import tokenizer
        tk = lambda x: '/'.join([w for w in tokenizer.load_spm(test_sentence_layers[k]['layer'].spm_path).EncodeAsPieces(x)])
        df[(k, 'Sentence')] = df[(k, 'Sentence')].map(tk)


swem_average_tokai_mecab
swem_ave_ft_tokai_spm_8000
swem_ave_gv_tokai_spm_8000
swem_average_tokai_spm_8000
swem_ave_ft_tokai_mecab


In [9]:
pd.merge(
    dfdict['swem_average_tokai_spm_8000'],
    dfdict['swem_ave_gv_tokai_spm_8000']
).style.apply(lambda x: coloring(0.2, x), axis=1)

In [1]:
for k, layer in test_sentence_layers.keys():
    print(k)
    pd.DataFrame([pd.DataFrame(pd.Series(arr.ravel()).describe()).transpose() for arr in layer.vecs])

NameError: name 'test_sentence_layers' is not defined

In [34]:
dfdict['swem_average_tokai_mecab']["名古屋市暴力団排除条例(23/231002/0999/Law(1))"]

,Sentence,Distance
1,この条例は、市における暴力団の排除の推進に関し、基本理念を定め、並びに市、市民及び事業者の責務を明らかにするとともに、市の施策の基本となる事項等を定めることにより、市、市民及び事業者が一体となって暴力団の排除を推進し、もって市民の安全で平穏な生活を確保し、及び社会経済活動の健全な発展に寄与することを目的とする。,0.055
2,この条例において、次の各号に掲げる用語の意義は、それぞれ当該各号に定めるところによる。,0
3,暴力団の排除は、暴力団が市民の生活、市内の事業活動及び青少年の健全な育成に不当な影響を及ぼす存在であることを認識した上で、暴力団を恐れないこと、暴力団を利用しないこと、暴力団に協力しないこと及び暴力団と交際しないことを基本として推進されなければならない。,0.012
4,暴力団の排除は、市、市民及び事業者が相互に連携し、及び協力して推進するものとする。,0.088
5,市は、前条に定める基本理念(以下「基本理念」という。)にのっとり、市民及び事業者の協力を得るとともに、愛知県(以下「県」という。)及び法第32条の2第1項の規定により愛知県暴力追放運動推進センターとして指定を受けた者その他の暴力団員による不当な行為の防止を目的とする団体(以下「推進センター等」という。)と連携を図りながら、暴力団の排除に関する施策を実施するものとする。,0.039
6,市は、県及び推進センター等と連携し、市民及び事業者が暴力団の排除に自主的に、かつ、相互に連携して取り組むことができるよう、市民及び事業者に対し、情報の提供その他の必要な支援を行うものとする。,0.169
7,事業者は、基本理念にのっとり、その行う事業により暴力団を利することとならないようにするとともに、市が実施する暴力団の排除に関する施策に協力しなければならない。,0.005
8,市民及び事業者は、暴力団の排除に資すると認められる情報を知ったときは、市、愛知県警察その他の関係行政機関に対し、当該情報を提供するよう努めなければならない。,0.082
9,市民は、基本理念にのっとり、暴力団の排除に自主的に、かつ、相互に連携して取り組むよう努めるとともに、市が実施する暴力団の排除に関する施策に協力するよう努めなければならない。,0.006
10,市民及び事業者は、暴力団の排除に資すると認められる情報を知ったときは、市、愛知県警察その他の関係行政機関に対し、当該情報を提供するよう努めなければならない。,0.092


In [22]:
threshold = 0.3
df = hmodel.comptable(testset, query, target_law_tags[1:3], 10, 'Sentence')
#display(df.style.apply(lambda x: coloring(d['threshold'], x), axis=1))
df

愛知県暴力団排除条例(23/230006/1813/Law(1))  \
                                                                                                                                                                                                                                                                                   Sentence   
1                                                                                                                         この条例は、暴力団の排除について、基本理念を定め、並びに県、事業者及び県民の責務を明らかにするとともに、県が実施する施策の基本となる事項等を定め、及び必要な規制を行うことにより、県、事業者及び県民が一体となって暴力団の排除を推進し、もって地域経済の健全な発展に寄与し、及び県民の安全で平穏な生活を確保することを目的とする。   
2                                                                                                                                                                                                                                               この条例において、次の各号に掲げる用語の意義は、それぞれ当該各号に定めるところによる。   
3                                                                                                                                                                                暴力団の排除は、暴力団が県内の事業活動及び県民の生活に不当な影響を与える存在であることを認識した上で、暴力団を利用しないこと、暴力団に協力しないこと及び暴力団と交際しないことを基本として推進されなければならない。   
4                                                                                                                                                                                                                                                          暴力団の排除は、県、事業者及び県民が協働して推進するものとする。   
5                                                                                                                                 県は、前条に定める基本理念(以下「基本理念」という。)にのっとり、法第三十二条の三第一項の規定により愛知県暴力追放運動推進センターとして指定を受けた者その他の暴力団員による不当な行為の防止を目的とする団体(以下「推進センター等」という。)と連携を図りながら、暴力団の排除に関する施策を実施する責務を有する。   
6                                                                                                                                                                                                                           県は、市町村が暴力団の排除に関する施策を実施する場合には、情報の提供、技術的な助言その他の必要な協力及び支援を行うものとする。   
7                                                                                                                                                                                                           事業者は、基本理念にのっとり、その行う事業により暴力団を利することとならないようにするとともに、県が実施する暴力団の排除に関する施策に協力しなければならない。   
8                                                                                                                                                                                                                                事業者は、暴力団の排除に資すると認められる情報を知ったときは、県に対し、当該情報を提供するよう努めなければならない。   
9                                                                                                                                                                                                    県民は、基本理念にのっとり、暴力団の排除に自主的に、かつ、相互に連携して取り組むよう努めるとともに、県が実施する暴力団の排除に関する施策に協力するよう努めなければならない。   
10                                                                                                                                                                                                                                県民は、暴力団の排除に資すると認められる情報を知ったときは、県に対し、当該情報を提供するよう努めなければならない。   
11                                                                                                                                                                                                                                              この条例の適用に当たっては、県民の権利を不当に侵害しないように留意しなければならない。   
12                                                                                                                                                                  県は、公共工事その他の県の事務又は事業により暴力団を利することとならないように、暴力団員又は暴力団若しくは暴力団員と密接な関係を有する者を公共工事の入札に参加させないことその他の暴力団の排除のために必要な措置を講ずるよう努めるものとする。   
13                                                                                              知事若しくは教育委員会又は指定管理者(地方自治法(昭和二十二年法律第六十七号)第二百四十四条の二第三項に規定する指定管理者をいう。)は、公の施設の利用の許可の申請があった場合において、当該公の施設が暴力団の活動に利用されることにより当該暴力団の利益になると認めるときは、当

In [10]:
query = next(ltag_gen)
k = 1
threshold = 0.3

print("query:", str(testset.get_elem(law_tags[i])), get_caption(query))
qlevel = re.split('\(', os.path.split(query)[1])[0]
pd.DataFrame(
    [( str(testset.get_elem(law_tags[i])), get_caption(law_tags[i]), round(s, 3), testset.get_elem(law_tags[i]).text) for i, s in zip(*list( law_index.knnQuery(hmodel.layers[qlevel][query], 20) ))],
    columns = ["name", "caption", "similarity", "text"]
)

NameError: name 'i' is not defined

In [11]:
k = 1
root_threshold =  1000
threshold = 0.3
print("query:", testset.get_elem(query))
qlevel = re.split('\(', os.path.split(query)[1])[0]
tidx, distance = law_index.knnQuery(hmodel.layers[qlevel][query], k=k+1)
target_law_tags = [query]
for i, dist in zip(tidx, distance):
    if law_tags[i] == query:
        continue
    if root_threshold and dist > root_threshold:
        print("not enough distance", testset.get_elem(law_tags[i]), sim)
        k -= 1
        continue
    print("add target:", testset.get_elem(law_tags[i]), sim)
    target_law_tags.append(law_tags[i])
for qi, q in enumerate(target_law_tags):
    df = hmodel.comptable(testset, q, target_law_tags[:qi]+target_law_tags[qi+1:], 10, 'Sentence')
    display(df.style.apply(lambda x: coloring(threshold, x), axis=1))

query: 愛知県公告式条例


NameError: name 'sim' is not defined

query: 議決事件指定条例
特別職の指定に関する条例 5.9604645e-08
議決事件指定条例 1.1920929e-07
沿道区域指定の基準に関する条例 1.1920929e-07
都市的施設その他の都市としての要件に関する条例 0.009170651
豊橋市新型インフルエンザ等対策本部条例 0.23891258
西尾市新型インフルエンザ等対策本部条例 0.23891258
職員の退職手当に関する条例の一部を改正する条例附則第二項の規定により退職手当の支給を受ける職員の範囲を定める規則 0.25446343
公立学校職員の退職手当に関する条例の一部を改正する条例附則第二項の規定により退職手当の支給を受ける職員の範囲を定める規則 0.26839417
南知多町における高齢者、障害者等の移動等の円滑化のために必要な特定公園施設の設置の基準に関する条例 0.31211913
武豊町移動等円滑化のために必要な特定公園施設の設置に関する基準を定める条例 0.31211913
尾張旭市指定地域密着型サービス事業者及び指定地域密着型介護予防サービス事業者の指定に関する基準を定める条例 0.3121192
尾張旭市道路構造の技術的基準を定める条例 0.3121192
尾張旭市市道に設ける案内標識等の寸法を定める条例 0.3121192
尾張旭市準用河川管理施設等の構造の技術的基準を定める条例 0.3121192
尾張旭市移動等円滑化のために必要な特定公園施設の設置に関する基準を定める条例 0.3121192
尾張旭市水道事業の布設工事監督者及び水道技術管理者に関する基準を定める条例 0.3121192
東郷町不育症治療費の助成に関する条例 0.3121192
東郷町移動等円滑化のために必要な特定公園施設の設置に関する基準を定める条例 0.3121192
南知多町水道事業の布設工事監督者及び水道技術管理者に関する条例 0.3121192
西尾市教育委員会公告式規則 0.32603073


In [5]:
with open("./../reikiset/21/210005/0067.xml") as f:
    print(f.read())

﻿<?xml version="1.0" encoding="UTF-8"?>
<Laws DictVer="">
<Law OriginalPromulgateDate="昭和二十四年三月二十八日条例第十四号" LawType="" Lang="ja" Year="" Era="" Num="">
	<LawNum>昭和二十四年三月二十八日条例第十四号</LawNum>
	<LawBody>
		<LawTitle>岐阜県公衆浴場法施行条例</LawTitle>
		<EnactStatement>昭和二十三年七月法律第百三十九号公衆浴場法により、岐阜県公衆浴場法施行条例を、次のように定める。</EnactStatement>
		<MainProvision>
			<Article Num="1">
				<ArticleTitle>第一条</ArticleTitle>
				<Paragraph Num="1">
					<ParagraphNum />
					<ParagraphSentence>
						<Sentence>公衆浴場法第二条第三項の規定による公衆浴場の設置の場所の配置の基準並びに同法第三条第二項の規定による公衆浴場の換気、採光、照明保温及び清潔その他入浴者の衛生及び風紀について必要な措置の基準は、この条例の定めるところによる。</Sentence>
					</ParagraphSentence>
				</Paragraph>
			</Article>
			<Article Num="2">
				<ArticleTitle>第二条</ArticleTitle>
				<Paragraph Num="1">
					<ParagraphNum />
					<ParagraphSentence>
						<Sentence Num="1" Function="Main">新たに設置しようとする公衆浴場は、既設の公衆浴場との間に、次に掲げる距離を保たなければならない。</Sentence>
						<Sentence Num="2" Function="Proviso">ただし、土地の状況、予想利用者数、人口密度等を考慮して公衆浴場の設置が公衆衛生上必要であると知事が認めたときは、この限りでない。</

In [ ]:
nagoya = '23/231002/0999/Law(1) 名古屋市暴力団排除条例'
aichi = '23/230006/1813/Law(1) 愛知県暴力団排除条例'
seto = '23/232041/0562/Law(1) 瀬戸市暴力団排除条例'

In [24]:
import sys 
import nmslib 
import time 
import math 
from scipy.sparse import csr_matrix 
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

In [25]:
# Read data points
testset.set_iterator_mode(level='Sentence', tag=True, sentence=False)
all_data_matrix =  np.matrix([hmodel.layers["Sentence"][tag] for tag in testset])

In [39]:
all_data_tags = {i:tag for i,tag in enumerate(testset)}

In [62]:
sample_data_matrix = all_data_matrix

In [27]:
# Create a held-out query data set
(data_matrix, query_matrix) = train_test_split(sample_data_matrix, test_size = 0.01)

In [63]:
query_matrix = np.matrix([query_vec])
data_matrix = sample_data_matrix
query_matrix.shape

(1, 500)

In [64]:
# Set index parameters
# These are the most important onese
M = 30
efC = 100

num_threads = 1
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}

In [65]:
# Number of neighbors 
K=100

In [66]:
# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='hnsw', space='cosinesimil') 
index.addDataPointBatch(data_matrix) 

610982

In [67]:
# Create an index
start = time.time()
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 30, 'indexThreadQty': 1, 'efConstruction': 100, 'post': 0}
Indexing time = 396.422498


In [97]:
%%timeit

index.createIndex(index_time_params) 

2.67 s ± 22.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [68]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params) 

Setting query-time parameters {'efSearch': 100}


In [112]:
data_matrix[1].tolist()[0] == index[0]

False

In [84]:
query_text = """動物の種類・発育状況に応じて、適正に餌・水を与えること。"""
vectorizer = hmodel.layers['Sentence']
vectorizer.model.load_wvmodel('/home/jovyan/results/hjst/models/wva_tokai_spm_16000/Sentence.lmodel/model_body/wvmodel.model')
tokenizer = spm.SentencePieceProcessor()
tokenizer.load('/home/jovyan/results/hjst/models/wva_tokai_spm_16000/Sentence.lmodel/spm/model.model')
query_vec = vectorizer.model._calc_vec(tokenizer.EncodeAsPieces(query_text))
query_matrix = np.matrix([query_vec])

In [85]:
# Querying
query_qty = query_matrix.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 


kNN time total=0.000645 (sec), per query=0.000645 (sec), per query adjusted for thread number=0.000645 (sec)


In [86]:
for rank, (idx, sim) in enumerate(zip(nbrs[0][0], nbrs[0][1])):
    print(rank, sim)
    elem = testset.get_elem(all_data_tags[idx])
    print(elem)
    print(elem.text)

0 0.249305
岡崎市動物の愛護及び管理に関する条例第2章第5条第1項第1号第1文
動物の種類、発育状況等に応じて適正に飼料及び水を給与すること。
1 0.329451
岡崎市動物の愛護及び管理に関する条例第2章第5条第1項第3号第1文
必要に応じて動物の種類、習性及び生理を考慮した飼養施設を設け、当該施設の設置に当たっては、日照、通風等の確保を図るなど適切な飼養環境を確保すること。
2 0.348658
豊橋市動物の愛護及び管理に関する条例第2章第5条第1項第1号第1文
適正に飼料及び水を与えること。
3 0.360377
豊橋市の政策推進における部等の役割を定める条例第3条第2項第4号3号細分第1文
水源林の涵養や新規水源の開発などにより、恒久・安定的な水資源の確保を図る。
4 0.376204
豊橋市動物の愛護及び管理に関する条例第2章第5条第1項第4号第1文
汚物及び汚水を適正に処理すること等により、飼養施設の内外を常に清潔に保ち、悪臭又は昆虫等の発生を防止すること。
5 0.37912
豊橋市の政策推進における部等の役割を定める条例第3条第2項第16号2号細分第1文
学校給食・学校保健管理、食育・食農教育の推進により、子どもの心と体の健康増進を図る。
6 0.381168
一宮市環境基本条例第2章第7条第1項第2号第1文
緑地、水辺等における自然環境を地域の自然的及び社会的条件に応じて体系的に保全すること。
7 0.38126
動物の愛護及び管理に関する条例第2章第5条第1項第1号第1文
適正にえさ及び水を与えること。
8 0.38126
名古屋市動物の愛護及び管理に関する条例第2章第5条第1項第1号第1文
適正にえさ及び水を与えること。
9 0.382779
扶桑町環境基本条例第2章第8条第1項第2号第1文
生態系の多様性の確保、野生生物の種の保存等が図られるとともに、緑地、水辺等における自然環境を地域の自然的及び社会的条件に応じて体系的に保全すること。
10 0.388206
豊橋市の政策推進における部等の役割を定める条例第3条第2項第14号1号細分第1文
水資源の効率的な利用に努めるとともに、水道施設の計画的な更新と耐震化を図ることにより、安全・安心な水道水の安定供給を確保する。
11 0.39446
岡崎市動物総合センター条例第4条第1項第1

In [57]:
nbrs

[(array([192,  93, 770, 950, 194, 193, 795, 876, 791, 942, 825,  55, 235,
         928, 745, 250, 929, 934,  32, 970, 847, 964, 215,  44, 833,  42,
         901, 884, 596, 217, 237, 941, 936, 939,  56, 492, 904, 919, 935,
         952,  67,  17, 864, 948, 883, 923,  97, 896, 828, 400, 885, 790,
         867, 890, 907, 849, 827, 151, 802, 945, 498, 910,  43, 906, 499,
         933, 848, 239,  95, 870, 826, 165, 959, 186, 900, 921, 241,  99,
         889, 119, 902,   2, 220,  52, 803, 188, 195, 958, 809, 189, 310,
         857, 865, 225, 829, 149, 937, 224,  71, 145], dtype=int32),
  array([ 0.32612216,  0.33281475,  0.37241876,  0.38690841,  0.38746548,
          0.39188552,  0.39448011,  0.40212572,  0.40290123,  0.42246723,
          0.42719483,  0.43361211,  0.43824816,  0.44252771,  0.44341385,
          0.45301402,  0.45506471,  0.46236753,  0.46460223,  0.46486294,
          0.4689579 ,  0.46993113,  0.4709689 ,  0.47818649,  0.48049474,
          0.4832263 ,  0.48854315,  0.49122

In [96]:
%%timeit
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)

1.2 s ± 6.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [103]:
# Computing gold-standard data 
print('Computing gold-standard data')

start = time.time()
sindx = NearestNeighbors(n_neighbors=K, metric='cosine', algorithm='brute').fit(data_matrix)
end = time.time()

print('Brute-force preparation time %f' % (end - start))

start = time.time() 
gs = sindx.kneighbors(query_matrix)
end = time.time()

print('brute-force kNN time total=%f (sec), per query=%f (sec)' % 
      (end-start, float(end-start)/query_qty) )

Computing gold-standard data
Brute-force preparation time 0.006200
brute-force kNN time total=1.866269 (sec), per query=0.000187 (sec)


In [94]:
%%timeit
sindx = NearestNeighbors(n_neighbors=K, metric='cosine', algorithm='brute').fit(data_matrix)

2.7 ms ± 18.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [95]:
%%timeit
gs = sindx.kneighbors(query_matrix)

1.12 s ± 36.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [104]:
# Finally computing recall
recall=0.0
for i in range(0, query_qty):
  correct_set = set(gs[1][i])
  ret_set = set(nbrs[i][0])
  recall = recall + float(len(correct_set.intersection(ret_set))) / len(correct_set)
recall = recall / query_qty
print('kNN recall %f' % recall)

kNN recall 0.988400


In [15]:
data_matrix.shape

(100, 500)

In [18]:
%timeit np.dot(query_matrix, data_matrix.T).max()

151 µs ± 139 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [17]:
%timeit np.argsort(np.dot(query_matrix[0], data_matrix.T))

31.8 µs ± 6.98 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [93]:
%%timeit
for q in query_matrix:
    arr = np.dot(q, data_matrix.T)
    arr.max()
    arr.max()

11.2 s ± 193 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
74.5 ms

0.033

In [23]:
30.6 ms

0.109923

In [24]:
183.5 ms

0.045714000000000005

In [ ]:
61.5 ms